In [1]:
!pip install llama-index
!pip install llama-index-core
!pip install --quiet transformers torch
!pip install llama-index-embeddings-openai
!pip install llama-index-llms-openai
!pip install llama-index-postprocessor-colbert-rerank

In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)

In [4]:
!mkdir -p 'data/paul_graham/'
!wget 'https://github.com/run-llama/llama_index/raw/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-03-27 10:22:35--  https://github.com/run-llama/llama_index/raw/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt [following]
--2024-03-27 10:22:36--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K

In [6]:
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-03-27 10:29:32--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.03s   

2024-03-27 10:29:32 (2.76 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [8]:
# import python dotenv
from dotenv import load_dotenv
import os

# load the .env file
load_dotenv()

True

In [11]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [12]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

# build index
index = VectorStoreIndex.from_documents(documents=documents)

In [13]:
from llama_index.postprocessor.colbert_rerank import ColbertRerank

colbert_reranker = ColbertRerank(
    top_n=5,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[colbert_reranker],
)
response = query_engine.query(
    "What did Sam Altman do in this essay?",
)

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [14]:
for node in response.source_nodes:
    print(node.id_)
    print(node.node.get_content()[:120])
    print("reranking score: ", node.score)
    print("retrieval score: ", node.node.metadata["retrieval_score"])
    print("**********")

48fc0edd-23e6-4a21-96e8-abccc1e7d6a2
This seemed strange advice, because YC was doing great. But if there was one thing rarer than Rtm offering advice, it wa
reranking score:  0.6401657462120056
retrieval score:  0.8212027754236421
**********
71a97d0a-0025-4ee2-b7ee-c65669ff7032
Why not organize a summer program where they'd start startups instead? We wouldn't feel guilty for being in a sense fake
reranking score:  0.6042895913124084
retrieval score:  0.8078020096082776
**********
e6baf3eb-d0db-46e0-9793-6d0df17ab593
What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and pro
reranking score:  0.5696992874145508
retrieval score:  0.7920438317553905
**********
0880a472-5095-4404-901c-e2d2f57f9c08
I knew that online essays would be a marginal medium at first. Socially they'd seem more like rants posted by nutjobs on
reranking score:  0.5589112639427185
retrieval score:  0.8091647704218679
**********
e30b22aa-8175-4d76-bb63-eafe

In [15]:
print(response)

Sam Altman was asked to become the president of Y Combinator (YC) after Paul Graham decided to step down from running YC. Initially, Sam Altman was hesitant as he wanted to start a startup to make nuclear reactors. However, after persistent efforts from Paul Graham, Sam Altman finally agreed to take over as the president of YC starting with the winter 2014 batch. Paul Graham gradually handed over the responsibilities of running YC to Sam Altman so he could learn the job, while Paul Graham focused on personal matters, such as his mother's illness.


In [16]:
response = query_engine.query(
    "Which schools did Paul attend?",
)

In [17]:
for node in response.source_nodes:
    print(node.id_)
    print(node.node.get_content()[:120])
    print("reranking score: ", node.score)
    print("retrieval score: ", node.node.metadata["retrieval_score"])
    print("**********")

2e0414a2-a712-41ce-9e95-7edfff661588
Grad students could take classes in any department, and my advisor, Tom Cheatham, was very easy going. If he even knew a
reranking score:  0.5909818410873413
retrieval score:  0.8060823726519702
**********
b06c49ce-4213-40e8-b05f-04f439cc554a
The foundation had merely been intro stuff, and the Accademia had been a (very civilized) joke. Now I was going to see w
reranking score:  0.5821996927261353
retrieval score:  0.8020923573546055
**********
f971d282-8d56-43f6-a9cc-fa19d871fa96
I've never known a teacher more beloved by her students. Large numbers of former students kept in touch with her, includ
reranking score:  0.5508488416671753
retrieval score:  0.7760295568936797
**********
e6baf3eb-d0db-46e0-9793-6d0df17ab593
What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and pro
reranking score:  0.5351229310035706
retrieval score:  0.781226579777117
**********
43f3d79b-95fc-4340-96a9-66090

In [18]:
print(response)

Paul attended RISD in the US and the Accademia di Belli Arti in Florence.
